In [2]:
import textwrap

# Expanded sample mapping to demonstrate full paths and tree rendering
mapping_text = textwrap.dedent("""
Folder map of c:\\Users\\dell\\Downloads\\Project
==================================================

├── root/
│   ├── src/
│   │   ├── lib/
│   │   └── utils/
│   ├── docs/
│   └── tests/
│       └── unit/
""").strip().splitlines()

# Settings
indent_unit = 4

# Data holders
nodes = []
empty_folders = set()
temp = None

# Helper: count leading spaces before any tree character
def count_indent(line):
    return len(line) - len(line.lstrip(' '))

# Process the mapping in one pass
for line_number, raw in enumerate(mapping_text):
    # skip header/separators
    if raw.startswith("Folder map") or raw.startswith("=") or not raw.strip():
        continue

    leading_spaces = count_indent(raw)
    # strip tree characters
    name = raw.lstrip(' ').lstrip('├── ').lstrip('│   ').lstrip('└── ')
    is_folder = name.endswith('/')

    # ID assignment
    node_id = nodes[-1]['id'] + 1 if nodes else 0

    # Parent and level logic
    if temp:
        if leading_spaces > temp['level'] * indent_unit:
            level = temp['level'] + 1
            parent_id = temp['id']
            empty_folders.discard(temp['id'])
        else:
            level = temp['level']
            parent_id = temp['parent_id'] if 'parent_id' in temp else None
        temp = None
    else:
        # Top-level or sibling
        level = 0
        parent_id = None

    # Create node record
    node = {
        'id': node_id,
        'name': name,
        'is_folder': is_folder,
        'parent_id': parent_id,
        'level': level
    }
    nodes.append(node)

    # Track empty folders & set temp
    if is_folder:
        empty_folders.add(node_id)
        temp = {'id': node_id, 'level': level, 'parent_id': parent_id}

# Helper: build full path by walking parent_id
def build_path(node):
    parts = []
    current = node
    while current:
        parts.append(current['name'].rstrip('/'))
        current = next((n for n in nodes if n['id'] == current['parent_id']), None)
    return '/'.join(reversed(parts)) + '/'

# Print empty folders list
print(f"Empty folders ({len(empty_folders)} total):")
for nid in sorted(empty_folders):
    print(f"– {build_path(nodes[nid])}")

# Print ascii tree for folders only
print("\nFolder-only tree:")
# Determine children for tree rendering
children = {n['id']: [] for n in nodes if n['is_folder']}
for n in nodes:
    if not n['is_folder'] and n['parent_id'] not in children:
        continue
    if n['parent_id'] is not None:
        children.setdefault(n['parent_id'], []).append(n)

def print_tree(node_id, prefix=""):
    node = next(n for n in nodes if n['id'] == node_id)
    marker = "(empty)" if node_id in empty_folders else ""
    print(f"{prefix}{node['name']} {marker}".rstrip())
    child_folders = [c for c in children.get(node_id, []) if c['is_folder']]
    for i, child in enumerate(child_folders):
        last = (i == len(child_folders) - 1)
        branch = "└── " if last else "├── "
        extension = "    " if last else "│   "
        print_tree(child['id'], prefix + branch)
        prefix = prefix.rstrip() + extension

# Find root nodes
root_nodes = [n for n in nodes if n['parent_id'] is None and n['is_folder']]
for root in root_nodes:
    print_tree(root['id'])


Empty folders (7 total):
– root/
– ├── src/
– ├── lib/
– utils/
– ├── docs/
– tests/
– unit/

Folder-only tree:
root/ (empty)
├── src/ (empty)
├── lib/ (empty)
utils/ (empty)
├── docs/ (empty)
tests/ (empty)
unit/ (empty)


In [ ]:

= textwrap.dedent("""
Folder map of c:\\Users\\dell\\Downloads\\与瑟夫
==================================================

├── .vscode/
│   └── metago_bookmarks.json
├── # ­ Active Malware or Exploits posing as.md
├── %9.js
├── 0_today.txt
""").strip().splitlines()
print(mapping_text);

['Folder map of c:\\Users\\dell\\Downloads\\与瑟夫', '==================================================', '', '├── .vscode/', '│   └── metago_bookmarks.json', '├── # \xad Active Malware or Exploits posing as.md', '├── %9.js', '├── 0_today.txt']


In [3]:
import os
import textwrap

def solve(mapping_input, indent_unit=4, output_path="output.txt"):
    """
    Parses a OneTab-style folder mapping (list of lines or filepath).
    Outputs an external text file with:
      1. List of empty folders with full paths.
      2. ASCII tree of folders, marking empties.
    """
    # Read lines
    if isinstance(mapping_input, str) and os.path.isfile(mapping_input):
        with open(mapping_input, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    elif isinstance(mapping_input, str):
        lines = mapping_input.splitlines()
    else:
        lines = mapping_input
    
    # Helpers
    def count_indent(line):
        return len(line) - len(line.lstrip(' '))
    
    def strip_markers(line):
        return line.lstrip(' ').lstrip('├── ').lstrip('│   ').lstrip('└── ')
    
    # Containers
    nodes = []         # list of nodes in order
    empty_folders = set()
    temp = None        # holds last dir info: {'id', 'level', 'parent_id'}
    
    # Single-pass parse
    for raw in lines:
        if raw.startswith("Folder map") or raw.startswith("=") or not raw.strip():
            continue
        
        indent = count_indent(raw)
        name = strip_markers(raw)
        is_folder = name.endswith('/')
        
        # ID assignment
        node_id = nodes[-1]['id'] + 1 if nodes else 0
        
        # Determine level & parent
        if temp:
            if indent > temp['level'] * indent_unit:
                level = temp['level'] + 1
                parent_id = temp['id']
                empty_folders.discard(temp['id'])
            else:
                level = temp['level']
                parent_id = temp['parent_id']
            temp = None
        else:
            # first or sibling at top level
            level = 0
            parent_id = None
        
        # Create node entry
        nodes.append({
            'id': node_id,
            'name': name,
            'is_folder': is_folder,
            'parent_id': parent_id,
            'level': level,
            'indent': indent
        })
        
        # Track empty folders and set temp for dirs
        if is_folder:
            empty_folders.add(node_id)
            temp = {'id': node_id, 'level': level, 'parent_id': parent_id}
    
    # Build full paths
    id_to_node = {n['id']: n for n in nodes}
    def build_path(node):
        parts = []
        cur = node
        while cur:
            parts.append(cur['name'].rstrip('/'))
            cur = id_to_node.get(cur['parent_id'])
        return '/'.join(reversed(parts)) + '/'
    
    # Prepare output lines
    out_lines = []
    out_lines.append(f"Empty folders ({len(empty_folders)} total):")
    for fid in sorted(empty_folders):
        out_lines.append(f"– {build_path(id_to_node[fid])}")
    out_lines.append("")  # blank line
    out_lines.append("Folder-only tree:")
    
    # Build children mapping
    children = {n['id']: [] for n in nodes if n['is_folder']}
    for n in nodes:
        pid = n['parent_id']
        if pid in children:
            children[pid].append(n)
    
    # Sort children by name
    for ch_list in children.values():
        ch_list.sort(key=lambda x: x['name'])
    
    # Recursive print helper
    def print_tree(node_id, prefix=""):
        node = id_to_node[node_id]
        mark = " (empty)" if node_id in empty_folders else ""
        out_lines.append(f"{prefix}{node['name']}{mark}")
        ch_list = [c for c in children.get(node_id, []) if c['is_folder']]
        for i, child in enumerate(ch_list):
            last = (i == len(ch_list)-1)
            branch = "└── " if last else "├── "
            ext = "    " if last else "│   "
            print_tree(child['id'], prefix + branch)
    
    # Print from roots
    roots = [n for n in nodes if n['parent_id'] is None and n['is_folder']]
    for root in roots:
        print_tree(root['id'])
    
    # Write to output file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(out_lines))
    
    return out_lines

# Example usage with string input
sample_input = textwrap.dedent("""
Folder map of c:\\Users\\dell\\Downloads\\与瑟夫
==================================================

├── .vscode/
│   └── metago_bookmarks.json
├── # ­ Active Malware or Exploits posing as.md
├── %9.js
├── 0_today.txt
""")
result = solve(sample_input, indent_unit=4, output_path='tree_output.txt')
#lines = solve("my_folder_map.txt", indent_unit=4, output_path="tree_out.txt")
    # Above is mentioned how you can use the file instead of the string

# Display result
for line in result:
    print(line)


Empty folders (1 total):
– .vscode/

Folder-only tree:
.vscode/ (empty)


In [4]:
# Updated solve function with emojis integrated

import os
import textwrap

def solve(mapping_input, indent_unit=4, output_path="output.txt"):
    """
    Parses a OneTab-style folder mapping (list of lines or filepath).
    Outputs an external text file with:
      1. List of empty folders with full paths, prefixed with 📁.
      2. ASCII tree of folders, marking empties with 📂.
    """
    # Read lines
    if isinstance(mapping_input, str) and os.path.isfile(mapping_input):
        with open(mapping_input, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    elif isinstance(mapping_input, str):
        lines = mapping_input.splitlines()
    else:
        lines = mapping_input
    
    # Helpers
    def count_indent(line):
        return len(line) - len(line.lstrip(' '))
    
    def strip_markers(line):
        return line.lstrip(' ').lstrip('├── ').lstrip('│   ').lstrip('└── ')
    
    # Containers
    nodes = []         # list of nodes in order
    empty_folders = set()
    temp = None        # holds last dir info: {'id', 'level', 'parent_id'}
    
    # Single-pass parse
    for raw in lines:
        if raw.startswith("Folder map") or raw.startswith("=") or not raw.strip():
            continue
        
        indent = count_indent(raw)
        name = strip_markers(raw)
        is_folder = name.endswith('/')
        
        # ID assignment
        node_id = nodes[-1]['id'] + 1 if nodes else 0
        
        # Determine level & parent
        if temp:
            if indent > temp['level'] * indent_unit:
                level = temp['level'] + 1
                parent_id = temp['id']
                empty_folders.discard(temp['id'])
            else:
                level = temp['level']
                parent_id = temp['parent_id']
            temp = None
        else:
            # first or sibling at top level
            level = 0
            parent_id = None
        
        # Create node entry
        nodes.append({
            'id': node_id,
            'name': name,
            'is_folder': is_folder,
            'parent_id': parent_id,
            'level': level,
            'indent': indent
        })
        
        # Track empty folders and set temp for dirs
        if is_folder:
            empty_folders.add(node_id)
            temp = {'id': node_id, 'level': level, 'parent_id': parent_id}
    
    # Build full paths
    id_to_node = {n['id']: n for n in nodes}
    def build_path(node):
        parts = []
        cur = node
        while cur:
            parts.append(cur['name'].rstrip('/'))
            cur = id_to_node.get(cur['parent_id'])
        return '/'.join(reversed(parts)) + '/'
    
    # Prepare output lines
    out_lines = []
    out_lines.append(f"📂 Empty folders ({len(empty_folders)} total):")
    for fid in sorted(empty_folders):
        out_lines.append(f"– 📁 {build_path(id_to_node[fid])}")
    out_lines.append("")  # blank line
    out_lines.append("📁 Folder-only tree:")
    
    # Build children mapping
    children = {n['id']: [] for n in nodes if n['is_folder']}
    for n in nodes:
        pid = n['parent_id']
        if pid in children:
            children[pid].append(n)
    
    # Sort children by name
    for ch_list in children.values():
        ch_list.sort(key=lambda x: x['name'])
    
    # Recursive print helper
    def print_tree(node_id, prefix=""):
        node = id_to_node[node_id]
        mark = " (empty)" if node_id in empty_folders else ""
        emoji = "📁" if node_id in empty_folders else "📂"
        out_lines.append(f"{prefix}{emoji} {node['name']}{mark}")
        ch_list = [c for c in children.get(node_id, []) if c['is_folder']]
        for i, child in enumerate(ch_list):
            last = (i == len(ch_list)-1)
            branch = "└── " if last else "├── "
            ext = "    " if last else "│   "
            print_tree(child['id'], prefix + branch)
    
    # Print from roots
    roots = [n for n in nodes if n['parent_id'] is None and n['is_folder']]
    for root in roots:
        print_tree(root['id'])
    
    # Write to output file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(out_lines))
    
    return out_lines

# Example usage with string input
sample_input = textwrap.dedent("""
Folder map of c:\\Users\\dell\\Downloads\\与瑟夫
==================================================

├── .vscode/
│   └── metago_bookmarks.json
├── # ­ Active Malware or Exploits posing as.md
├── %9.js
├── 0_today.txt
""")
result = solve(sample_input, indent_unit=4, output_path='tree_output_with_emojis.txt')

# Display result
for line in result:
    print(line)


📂 Empty folders (1 total):
– 📁 .vscode/

📁 Folder-only tree:
📁 .vscode/ (empty)
